In [1]:
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
(X_train,y_train), (X_test,y_test) = boston_housing.load_data() 

65536/57026 [==================================] - 0s 0us/step


In [6]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [34]:
input_shape = X_train.shape[1]
output_shape = y_train.shape[1]
model = Sequential()
model.add(Dense(28,activation='relu',input_shape=(input_shape,)))
model.add(Dense(5,activation='relu'))
model.add(Dense(output_shape))

In [35]:
opt = 'rmsprop'
loss = 'mse'
metrics = ['mse']
model.compile(optimizer=opt, loss = loss, metrics = metrics)

In [36]:
epochs = 30
batch_size = 20
model.fit(X_train,
          y_train,
          epochs=epochs,
          batch_size=batch_size,
         validation_data=(X_test,y_test))

Epoch 1/30
21/21 [==============================] - 0s 5ms/step - loss: 277.2371 - mse: 277.2371 - val_loss: 112.1641 - val_mse: 112.1641
Epoch 2/30
21/21 [==============================] - 0s 1ms/step - loss: 110.8949 - mse: 110.8950 - val_loss: 232.6753 - val_mse: 232.6753
Epoch 3/30
21/21 [==============================] - 0s 2ms/step - loss: 104.1983 - mse: 104.1983 - val_loss: 94.9543 - val_mse: 94.9543
Epoch 4/30
21/21 [==============================] - 0s 2ms/step - loss: 91.4415 - mse: 91.4415 - val_loss: 76.7836 - val_mse: 76.7836
Epoch 5/30
21/21 [==============================] - 0s 2ms/step - loss: 80.3712 - mse: 80.3712 - val_loss: 107.0195 - val_mse: 107.0195
Epoch 6/30
21/21 [==============================] - 0s 2ms/step - loss: 76.8424 - mse: 76.8424 - val_loss: 80.6571 - val_mse: 80.6571
Epoch 7/30
21/21 [==============================] - 0s 2ms/step - loss: 72.0099 - mse: 72.0099 - val_loss: 89.9515 - val_mse: 89.9515
Epoch 8/30
21/21 [==============================] 

In [37]:
np.sqrt(model.evaluate(X_test,y_test)[0])

4/4 [==============================] - 0s 1ms/step - loss: 71.6031 - mse: 71.6031


8.461859458223273

In [17]:
lr_model = LinearRegression()
lr_model.fit(X_train,y_train)

LinearRegression()

In [20]:
np.sum((lr_model.predict(X_test) - y_test)**2)

2365.951124155147

In [22]:
lr_model.score(X_test,y_test)

0.7213535934621549

In [26]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train,y_train)

/tmp/ipykernel_12749/2971716159.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train,y_train)


RandomForestRegressor()

In [27]:
np.sum((rf_model.predict(X_test) - y_test)**2)/y_test.size

15153.752135372557

In [33]:
np.sqrt(np.sum((model.predict(X_test) - y_test)**2)/y_test.size)

8.329958301447853

In [3]:
from keras.datasets import reuters

In [4]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=10000)

In [5]:
X = np.concatenate([X_train,X_test])
y = np.concatenate([y_train,y_test])

In [6]:
word_index = reuters.get_word_index()

In [7]:
index_word = { v:k for k, v in word_index.items()}

In [8]:
def make_sentence(x):
    doc = []
    for i in x:
        tmp = []
        for j in range(len(i)):
            if i[j] < 30980:
                tmp.append(index_word[i[j]])
                sentence = ' '.join(tmp)
        doc.append(sentence)
    return pd.DataFrame(doc,columns=['text'])

In [9]:
X = make_sentence(X)

In [10]:
xx = []
for i in X.text.str.split(' '):
    xx.extend(i)

In [11]:
aa = np.unique(np.array(xx))

In [12]:
word_index1 = { k:v for k, v in enumerate(list(aa))}

In [13]:
index_word1 = { v:k for k, v in word_index1.items()}

In [14]:
len(X)

11228

In [15]:
def make_index(x):
    x = x.split(' ')
    tmp = []
    for i in x: 
        tmp.append(index_word1[i])
    return tmp

In [16]:
X = X.text.apply(make_index)

In [17]:
def vectorize_sequences(sequences, dimension=10000):
    # 크기가 (len(sequences), dimension))이고 모든 원소가 0인 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # results[i]에서 특정 인덱스의 위치를 1로 만듭니다
    return results

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2)

In [22]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [23]:
input_shape = X_train.shape[1]
output_shape = y_train.shape[1]

In [26]:
model = Sequential()
model.add(Dense(1024,activation='relu',input_shape=(input_shape,)))
model.add(Dense(256,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(1024,activation='relu'))
model.add(Dense(output_shape,activation='softmax'))

2022-05-02 17:23:30.678696: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 17:23:30.679952: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [27]:
opt = 'adam'
loss = 'categorical_crossentropy'
metrics = ['accuracy']
model.compile(optimizer=opt,
             loss=loss,
             metrics=metrics)

In [28]:
epochs = 100
batch_size = 400
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

2022-05-02 17:23:32.990535: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
23/23 [==============================] - 5s 220ms/step - loss: 1.9906 - accuracy: 0.5640
Epoch 2/100
23/23 [==============================] - 5s 210ms/step - loss: 1.0379 - accuracy: 0.7497
Epoch 3/100
23/23 [==============================] - 5s 204ms/step - loss: 0.6323 - accuracy: 0.8417
Epoch 4/100
23/23 [==============================] - 5s 202ms/step - loss: 0.3897 - accuracy: 0.9024
Epoch 5/100
23/23 [==============================] - 5s 205ms/step - loss: 0.2604 - accuracy: 0.9318
Epoch 6/100
23/23 [==============================] - 5s 205ms/step - loss: 0.1804 - accuracy: 0.9482
Epoch 7/100
23/23 [==============================] - 5s 202ms/step - loss: 0.1425 - accuracy: 0.9538
Epoch 8/100
23/23 [==============================] - 5s 200ms/step - loss: 0.1214 - accuracy: 0.9575
Epoch 9/100
23/23 [==============================] - 5s 200ms/step - loss: 0.1079 - accuracy: 0.9551
Epoch 10/100
23/23 [==============================] - 5s 199ms/step - loss: 0.1032 - accura

KeyboardInterrupt: 

In [79]:
X_test[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [45]:
data = '''
 decision came after Germany modern pentathlon coach Kim Raisner struck a horse at the Tokyo Games last year when it refused to jump a fence, an incident that led to widespread criticism of the sport.

UIPM, which is set to hold an executive board meeting later on Monday, did not immediately respond to a Reuters request for comment.

The report also said Britain Olympic champion Joe Choong was among athletes who wrote to International Olympic Committee President Thomas Bach on Sunday asking him to intervene.

The letter stated that athletes had not played a central role in the review and consultation process for the fifth discipline, as per the IOC's expectations, the report added.

The IOC was also made aware of a poll by Pentathlon United - a group formed to block the removal of the equestrian element - which showed 95% of athletes were unhappy with the UIPM decision, the report said.

Modern pentathlon has been an Olympic sport since 1912 and features five events: fencing, swimming, equestrian show jumping, pistol shooting and cross-country running.
'''

In [46]:
data = data.replace('\n','')

In [51]:
def make_index(x):
    x = x.split(' ')
    tmp = []
    for i in x:
        try:
            tmp.append(index_word1[i])
        except KeyError:
            tmp.append(7)
    return tmp

In [54]:
def vectorize_sequences(sequences, dimension=10000):
    # 크기가 (len(sequences), dimension))이고 모든 원소가 0인 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # results[i]에서 특정 인덱스의 위치를 1로 만듭니다
    return results
np.argmax(model.predict(vectorize_sequences(make_index(data))))

647

In [92]:
x = make_index(data)
t = np.zeros(10000)
for i in range(len(x)):
    t[x[i]] = 1

In [96]:
np.argmax(model.predict(t.reshape(1,-1)))

3

In [81]:
model.predict(vectorize_sequences(make_index(data[0]))).shape

(2, 46)

In [77]:
vectorize_sequences(make_index(data))

(172, 10000)

In [71]:
np.argmax(model.predict(vectorize_sequences(make_index(data))),axis=1)

array([ 3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  4,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3, 20,  3,  3,  3,  3,  3,  3,  3, 19,  3,  3, 16,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,  3, 20,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3])